This notebook gathers prediction results produced without missing values

In [6]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
#save and load function
import pickle
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
  

In [8]:
test_goal = pd.read_csv("sample_submission.csv")

In [9]:
test_goal.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [10]:
test = pd.read_csv("test.csv")
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [11]:
test_target = test.set_index(["shop_id","item_id"])

In [12]:
test_target.head()

ID
shop_id item_id    
5       5037      0
        5320      1
        5233      2
        5232      3
        5268      4

In [13]:
#group by item id from the prediction

In [14]:
#load the test
test_pred_df = load_obj("data/test_pred_df_mlp_complete")

In [15]:
test_pred_df.shop_id = test_pred_df.shop_id.astype(int)
test_pred_df.item_id = test_pred_df.item_id.astype(int)

In [16]:
test_pred = test_pred_df.copy()

In [18]:
test_pred.head()

,shop_id,item_id,item_category_id,top_categories_id,mlp_pred_block
0,0,30,40.0,13.0,0.164362
2393,0,14798,40.0,13.0,0.170736
2394,0,14799,37.0,13.0,0.170736
2395,0,14800,40.0,13.0,0.170736
2396,0,14804,37.0,13.0,0.170736


In [19]:
test_pred = test_pred.set_index(["shop_id","item_id"])

In [20]:
test_pred.head()

item_category_id  top_categories_id  mlp_pred_block
shop_id item_id                                                     
0       30                   40.0               13.0        0.164362
        14798                40.0               13.0        0.170736
        14799                37.0               13.0        0.170736
        14800                40.0               13.0        0.170736
        14804                37.0               13.0        0.170736

In [21]:
test_merge = test_target.join(test_pred,how = "left")
max(test_merge["mlp_pred_block"])

0.17073566346872632

In [22]:
#test_pred = test_pred.reset_index()
test_pred.head()

item_category_id  top_categories_id  mlp_pred_block
shop_id item_id                                                     
0       30                   40.0               13.0        0.164362
        14798                40.0               13.0        0.170736
        14799                37.0               13.0        0.170736
        14800                40.0               13.0        0.170736
        14804                37.0               13.0        0.170736

In [23]:
test_merge.head()

ID  item_category_id  top_categories_id  mlp_pred_block
shop_id item_id                                                         
5       5037      0              19.0                6.0        0.170736
        5320      1              55.0               15.0        0.170736
        5233      2              19.0                6.0        0.170736
        5232      3              23.0                6.0        0.170736
        5268      4              20.0                6.0        0.170736

In [24]:
#check NAs 
test_merge["mlp_pred_block"].isnull().unique()

array([False])

In [25]:
test_merge = test_merge.reset_index()

In [26]:
result = test_merge[["ID","mlp_pred_block"]]
result.columns = ["ID","item_cnt_month"]

In [27]:
#write to a file
result.to_csv("result_MLP_complete.csv",index = False)

##### LSTM results

In [28]:
#load LSTM train for column information
X_train = load_obj("data/X_train_LSTM_complete")

##### prediction using cnt+price+seasonal effect for 12 months window

In [29]:
X_train_sub = X_train[["shop_id","item_id","item_category_id","top_categories_id"]]

In [30]:
#load the test
#prediction using cnt+price+seasonal effect for 12 months window
test_pred_df = load_obj("data/y_test_pred_cnt_complete_price_cnt_12")

In [31]:
X_train_sub["prediction"] = test_pred_df

C:\Users\Zijun\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
X_train_sub.head()

,shop_id,item_id,item_category_id,top_categories_id,prediction
0,0.0,30.0,40.0,13.0,-3.230525e-08
2393,0.0,14798.0,40.0,13.0,1.108856e-08
2394,0.0,14799.0,37.0,13.0,-1.059379e-08
2395,0.0,14800.0,40.0,13.0,-1.003500e-07
2396,0.0,14804.0,37.0,13.0,8.795178e-08


In [33]:
test_pred_df = X_train_sub.copy()

In [34]:
test_pred_df.shop_id = test_pred_df.shop_id.astype(int)
test_pred_df.item_id = test_pred_df.item_id.astype(int)

In [35]:
#set index
test_pred_df = test_pred_df.set_index(["shop_id","item_id"])

In [36]:
#join tables
test_merge = test_target.join(test_pred_df,how = "left")
max(test_merge["prediction"])

21.07992172241211

In [37]:
test_merge = test_merge.reset_index()

In [38]:
result = test_merge[["ID","prediction"]]
result.columns = ["ID","item_cnt_month"]

In [39]:
#write to a file
result.to_csv("result_LSTM_12month_cnt_price_season.csv",index = False)

##### everything and 3 months windows

In [40]:
#load the test
X_train_sub = X_train[["shop_id","item_id","item_category_id","top_categories_id"]]
test_pred_df = load_obj("data/y_test_pred_complete_everything_3month")

In [41]:
X_train_sub["prediction"] = test_pred_df

C:\Users\Zijun\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
test_pred_df = X_train_sub.copy()
test_pred_df.shop_id = test_pred_df.shop_id.astype(int)
test_pred_df.item_id = test_pred_df.item_id.astype(int)

#set index
test_pred_df = test_pred_df.set_index(["shop_id","item_id"])

In [43]:
#join tables
test_merge = test_target.join(test_pred_df,how = "left")
max(test_merge["prediction"])

34.06584167480469

In [44]:
test_merge = test_merge.reset_index()

In [45]:
result = test_merge[["ID","prediction"]]
result.columns = ["ID","item_cnt_month"]

In [46]:
#write to a file
result.to_csv("result_LSTM_3month_everything.csv",index = False)

##### with selective features: cnt,price,seasonal effect, shop id, top categories, for 4 months window

In [47]:
#load the test
X_train_sub = X_train[["shop_id","item_id","item_category_id","top_categories_id"]]
test_pred_df = load_obj("data/y_test_pred_complete_cnt_price_seasonal_shop_cat1_4month")

In [48]:
X_train_sub["prediction"] = test_pred_df

C:\Users\Zijun\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
test_pred_df = X_train_sub.copy()
test_pred_df.shop_id = test_pred_df.shop_id.astype(int)
test_pred_df.item_id = test_pred_df.item_id.astype(int)

#set index
test_pred_df = test_pred_df.set_index(["shop_id","item_id"])

In [50]:
#join tables
test_merge = test_target.join(test_pred_df,how = "left")
max(test_merge["prediction"])

11.047061920166016

In [51]:
test_merge = test_merge.reset_index()

In [52]:
result = test_merge[["ID","prediction"]]
result.columns = ["ID","item_cnt_month"]

In [53]:
#write to a file
result.to_csv("result_LSTM_4month_selective.csv",index = False)

In [54]:
result["item_cnt_month"].isnull().unique()

array([False])